In [1]:
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet_v2 import preprocess_input
import numpy as np

In [3]:
filepath=r'C:\Users\shruti and rahul\Desktop\Python\Python course\Projects\Realtime Object detection\Real time detection\haarcascade_frontalface_alt2.xml'
model=load_model('mask_detector.h5')
detector=cv2.CascadeClassifier(filepath)

In [4]:
label_dict={0:'Mask',1:'No Mask'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [8]:
video_capture=cv2.VideoCapture(0)

while True:
    ret, frame=video_capture.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=detector.detectMultiScale(gray,1.1,5)
    
    for (x,y,w,h) in faces:
        face_extracted=frame[y:y+h,x:x+w]
        face_extracted=cv2.resize(face_extracted,(224,224))
        face_extracted=img_to_array(face_extracted)
        face_extracted=np.expand_dims(face_extracted,axis=0)
        face_extracted=preprocess_input(face_extracted)
        preds=model.predict(face_extracted)
        
        label=np.argmax(preds,axis=1)[0]

        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame, label_dict[label], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
        
    cv2.imshow('Video',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()